# filter ALL simulations and save result

In [1]:
import numpy as np
import os, glob
import matplotlib.pyplot as plt
plt.rc("font",size=14)

from netCDF4 import Dataset
import xarray, wrf

from pandas import to_datetime, date_range, Timedelta
from scipy.signal import welch
from matplotlib.cm import get_cmap
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature

In [2]:
plt.rc("font",size=14)

In [3]:
from scipy import fftpack

In [4]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
height = 100.0

In [13]:
dx = {"MYNN":333.0,
       "SH":333.0,
       "VLES":333.0,
       "YSU":333.0,
       "LES_25":25.0,
       "LES_100":100.0,
       "VLES-ISO":333.0}

In [34]:
day      = 21

In [35]:
directories = {"VLES-ISO":"/glade/scratch/doubrawa/VLES/03{0}15-Isotropic".format(day)}
#                "MYNN":"/glade/scratch/doubrawa/MYNN/03{0}15/".format(day),
#                "SH":"/glade/scratch/doubrawa/SH/03{0}15/".format(day),
#                "VLES":"/glade/scratch/doubrawa/VLES/03{0}15/".format(day),
#                "YSU":"/glade/scratch/doubrawa/YSU/03{0}15/".format(day),
#                "LES_25":"/glade/scratch/doubrawa/postProcessing/",
#                "LES_100":"/glade/scratch/doubrawa/postProcessing/"}

In [36]:
figPath = "/glade/u/home/doubrawa/figs/"

# read in all the data

In [37]:
t_start = to_datetime("2015-03-{0} 14:00:00".format(day),  format="%Y-%m-%d %H:%M:%S")
t_end   = to_datetime("2015-03-{0} 00:00:00".format(day+1),format="%Y-%m-%d %H:%M:%S")
t_vec   = date_range(start=t_start, end=t_end, freq='30min')
dt      = {"LES_25":10,"LES_100":60,"MYNN":30*60,"YSU":30*60,"SH":30*60,"VLES":30*60,"VLES-ISO":30*60}

In [38]:
w     = {}
t     = {}
xlat  = {}
xlong = {}

In [39]:
for key,directory in directories.items():
    print(key)
    for datetime in t_vec:
        if not( (datetime.day==28) & (datetime.hour==19) & (datetime.minute==30) ):
            datetime_str  = "{0:%Y-%m-%d_%H:%M}".format(datetime)       
            print(datetime)
            if "LES_" not in key:
                wrfout        = glob.glob(os.path.join(directory,'wrfout_d03*{0}*'.format(datetime_str)))[0]            
                wrfnc_for_wrf = Dataset(wrfout,'r')
                wrfnc         = xarray.open_dataset(wrfout)
                wrf_datetime  = to_datetime(wrfnc.XTIME[0].data)

                hgt           = wrf.getvar(wrfnc_for_wrf, "ter", units="m")
                z             = wrf.getvar(wrfnc_for_wrf, "z", units="m")
                zref          = z-hgt
                w_3d          = wrf.getvar(wrfnc_for_wrf, "wa", units="m s-1")
                w_2d          = wrf.interplevel(w_3d, zref, height, meta=True)    
                t_3d          = wrf.getvar(wrfnc_for_wrf, "theta", units="K")
                t_2d          = wrf.interplevel(t_3d, zref, height, meta=True)    
                lat           = wrf.getvar(wrfnc_for_wrf, "lat")
                lon           = wrf.getvar(wrfnc_for_wrf, "lon")                
                w_2d["time"]  = w_2d["Time"]
                w_2d          = w_2d.expand_dims('time')            
                t_2d["time"]  = t_2d["Time"]
                t_2d          = t_2d.expand_dims('time')                                          
            else:
                wrfout       = sorted(glob.glob(os.path.join(directory,'WRF_{0}m_{1:%Y-%m-%d}*{1:%H:%M}*.nc'.format(key,datetime))))
                if len(wrfout)==0:
                    wrfout       = sorted(glob.glob(os.path.join(directory,'WRF_{0}m_{1:%Y-%m-%d}*{1:%H:%M}*.nc'.format(key,datetime+Timedelta("1min")))))                
                if len(wrfout)==0:
                    break
                else:
                    wrfout        = wrfout[0]
                wrfnc         = xarray.open_dataset(wrfout)
                wrf_datetimes = [to_datetime(i) for i in wrfnc.time.data]
                min_diff      = np.min([(i - datetime).seconds for i in wrf_datetimes])
                min_diff_idx  = np.argmin([(i - datetime).seconds for i in wrf_datetimes])            

                condition = min_diff <= dt[key]
                if not(condition):
                    print ("ERROR")
                    print (datetime)
                    print (wrf_datetimes[min_diff_idx])
                    break            
                wrfnc  = wrfnc.isel(time=min_diff_idx)
                w_2d = wrfnc["w"]
                t_2d = wrfnc["theta"]
                lat  = wrfnc.xlat
                lon  = wrfnc.xlong

            if datetime==t_vec[0]:
                w[key]     = w_2d.copy()
                t[key]     = t_2d.copy()
                xlat[key]  = lat
                xlong[key] = lon
            else:
                w[key] = xarray.concat([w[key], w_2d], dim='time')
                t[key] = xarray.concat([t[key], t_2d], dim='time') 

VLES-ISO
2015-03-21 14:00:00
2015-03-21 14:30:00
2015-03-21 15:00:00
2015-03-21 15:30:00
2015-03-21 16:00:00
2015-03-21 16:30:00
2015-03-21 17:00:00
2015-03-21 17:30:00
2015-03-21 18:00:00
2015-03-21 18:30:00
2015-03-21 19:00:00
2015-03-21 19:30:00
2015-03-21 20:00:00
2015-03-21 20:30:00
2015-03-21 21:00:00
2015-03-21 21:30:00
2015-03-21 22:00:00
2015-03-21 22:30:00
2015-03-21 23:00:00
2015-03-21 23:30:00
2015-03-22 00:00:00


# get equivalent x/y coordinates across domains

In [40]:
ref_wrfnc = Dataset("/glade/scratch/domingom/Cheyenne/XPIA_mesoLES/SIMULS/WRF_mesoLES_4dom_RAP_2015_03_12_mesoLES/HOUR_14_1/wrfout_d04_2015-03-13_14:00:10_0000")

In [41]:
x = {}
y = {}
for key in directories.keys():

    x[key], y[key] = wrf.ll_to_xy(ref_wrfnc, xlat[key], xlong[key])
    x[key]   = np.reshape(x[key].data,w[key].shape[1:])
    y[key]   = np.reshape(y[key].data,w[key].shape[1:])        

# clip all of them to the same area

In [42]:
xmin, xmax = (0,1200)#np.min(x["LES_25"]), np.max(x["LES_25"])
ymin, ymax = (0,1200)#np.min(y["LES_25"]), np.max(y["LES_25"])       

In [43]:
w_clipped   = {}
t_clipped   = {}

for key in directories.keys():
    
    if key!="LES_25":

        print(key)

        condition_x = ( (x[key] >= xmin) & (x[key]<=xmax) )
        condition_y = ( (y[key] >= ymin) & (y[key]<=ymax) )
        condition   = condition_x & condition_y

        idx_sn, idx_we = np.where(condition)
        idx_sn         = np.unique(idx_sn)
        idx_we         = np.unique(idx_we)

        w_clipped[key] = (w[key].isel(west_east=idx_we,south_north=idx_sn)).copy()
        t_clipped[key] = (t[key].isel(west_east=idx_we,south_north=idx_sn)).copy()

        w_clipped[key]["west_east"]   = (x[key][0,:][idx_we]).copy()
        w_clipped[key]["south_north"] = (y[key][:,0][idx_sn]).copy()
        t_clipped[key]["west_east"]   = (x[key][0,:][idx_we]).copy()
        t_clipped[key]["south_north"] = (y[key][:,0][idx_sn]).copy()
    
if "LES_25" in directories.keys():
    key="LES_25"
    print(key)
    w_clipped[key]   = w[key]   
    t_clipped[key]   = t[key]   
    w_clipped[key]["west_east"]   = x[key][0,:].copy()
    w_clipped[key]["south_north"] = y[key][:,0].copy()
    t_clipped[key]["west_east"]   = x[key][0,:].copy()
    t_clipped[key]["south_north"] = y[key][:,0].copy()

VLES-ISO


# apply filter

In [44]:
k_cut = 1/(333*4.)    

In [45]:
w_filt = {}
t_filt = {}
for key in w_clipped.keys():
    n_t  = len(w_clipped[key].time)
    n_we = len(w_clipped[key].west_east)
    n_sn = len(w_clipped[key].south_north)
    w_filt[key] = np.zeros((n_t,n_sn,n_we))
    t_filt[key] = np.zeros((n_t,n_sn,n_we))

In [46]:
for key in w_clipped.keys():
    print (key)
    n_t  = len(w_clipped[key].time)
    for i_t in range(n_t):
        for pair in [(w_clipped,w_filt),(t_clipped,t_filt)]:
            clipped, filtered      = pair
            this_time              = clipped[key].interpolate_na(dim='south_north').interpolate_na(dim='west_east').data[i_t,:,:].copy()                
            psd_2d                 = fftpack.fft2(this_time)
            kappas_1d              = fftpack.fftfreq(len(this_time),d=dx[key])
            [kappas_j,kappas_i]    = np.meshgrid(kappas_1d,kappas_1d)   
            idx                    = np.where( (np.abs(kappas_j)>k_cut) | (np.abs(kappas_i)>k_cut) )
            psd_2d_filt            = psd_2d.copy()
            psd_2d_filt[idx]       = 0.0+0j
            filtered[key][i_t,:,:] = fftpack.ifft2(psd_2d_filt).real     

VLES-ISO


# turn filtered data into dataarray

In [47]:
for key in w_filt.keys():
    print (key)

    if "LES_" not in key:
        w_clipped[key]["lat"] = w_clipped[key]["XLAT"]
        w_clipped[key]["lon"] = w_clipped[key]["XLONG"] 
        w_clipped[key]        = w_clipped[key].drop("XTIME").drop("Time").drop("XLAT").drop("XLONG")
        t_clipped[key]["lat"] = t_clipped[key]["XLAT"]
        t_clipped[key]["lon"] = t_clipped[key]["XLONG"]        
        t_clipped[key]        = t_clipped[key].drop("XTIME").drop("Time").drop("XLAT").drop("XLONG")
    else:
        w_clipped[key]["lat"] = w_clipped[key]["xlat"]
        w_clipped[key]["lon"] = w_clipped[key]["xlong"] 
        w_clipped[key]        = w_clipped[key].drop("xlat").drop("xlong")
        t_clipped[key]["lat"] = t_clipped[key]["xlat"]
        t_clipped[key]["lon"] = t_clipped[key]["xlong"]        
        t_clipped[key]        = t_clipped[key].drop("xlat").drop("xlong")    
    
    w_filt[key] = xarray.DataArray(w_filt[key],
                     coords={"time":w_clipped[key].time,
                             "south_north":w_clipped[key].south_north,
                             "west_east":w_clipped[key].west_east,
                             "lat":w_clipped[key].lat,
                             "lon":w_clipped[key].lon}, 
                     dims=("time","south_north","west_east"), 
                     name="w (filtered for k < 1/{0:.0f}".format(1/k_cut), 
                     attrs={"unit":"m s-1","stagger":""})
    
    t_filt[key] = xarray.DataArray(t_filt[key],
                     coords={"time":t_clipped[key].time,
                             "south_north":t_clipped[key].south_north,
                             "west_east":t_clipped[key].west_east,
                             "lat":t_clipped[key].lat,
                             "lon":t_clipped[key].lon}, 
                     dims=("time","south_north","west_east"), 
                     name="potential temperature (filtered for k < 1/{0:.0f}".format(1/k_cut), 
                     attrs={"unit":"K","stagger":""})

VLES-ISO


# put dataarrays together into a dataset

In [48]:
for key in w_clipped.keys():
    if "projection" in w_clipped[key].attrs.keys():
        print (key)
        w_clipped[key].attrs = w_filt[key].attrs
        t_clipped[key].attrs = t_filt[key].attrs

VLES-ISO


In [49]:
for key in w_clipped.keys():
    print (key)
    dataset = xarray.Dataset(data_vars={"w":w_clipped[key],
                          "w_filt":w_filt[key],
                          "theta":t_clipped[key],
                          "theta_filt":t_filt[key]})
 
    dataset.assign_coords(z=100.0)
    pathout = os.path.join("/glade/scratch/doubrawa/postProcessing/","WRF_{0}_{1:%Y-%m-%d}_FILTERED.nc".format(key,t_start))
    dataset.to_netcdf(pathout)

VLES-ISO
